# Interacting with CLIP

[CLIP paper](https://arxiv.org/abs/2103.00020)

![CLIP objective](https://github.com/openai/CLIP/raw/main/CLIP.png)

This is a self-contained notebook that shows how to download and run CLIP models, calculate the similarity between arbitrary image and text inputs, and perform zero-shot image classifications.

# Preparation for Colab

Make sure you're running a GPU runtime; if not, select "GPU" as the hardware accelerator in Runtime > Change Runtime Type in the menu. The next cells will install the `clip` package and its dependencies, and check if PyTorch 1.7.1 or later is installed.

In [ ]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

In [ ]:
import numpy as np
import torch
from pkg_resources import packaging

print("Torch version:", torch.__version__)


# Loading the model

`clip.available_models()` will list the names of available CLIP models.

In [ ]:
import clip

clip.available_models()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model, preprocess = clip.load("ViT-B/16")
model = model.to(device).eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

# Image Preprocessing

We resize the input images and center-crop them to conform with the image resolution that the model expects. Before doing so, we will normalize the pixel intensity using the dataset mean and standard deviation.

The second return value from `clip.load()` contains a torchvision `Transform` that performs this preprocessing.



In [ ]:
preprocess

# Text Preprocessing

We use a case-insensitive tokenizer, which can be invoked using `clip.tokenize()`. By default, the outputs are padded to become 77 tokens long, which is what the CLIP models expects.

In [ ]:
clip.tokenize("Hello World!")

# Setting up input images and texts

We are going to feed 8 example images and their textual descriptions to the model, and compare the similarity between the corresponding features.

The tokenizer is case-insensitive, and we can freely give any suitable textual descriptions.

In [ ]:
import os
import skimage
from skimage import data
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from collections import OrderedDict
import torch

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# images in skimage to use and their textual descriptions
descriptions = {
    "page": "a page of text about segmentation",
    "cat": "a facial photo of a tabby cat",
    "astronaut": "a portrait of an astronaut with the American flag",
    "rocket": "a rocket standing on a launchpad",
    "camera": "a person looking at a camera on a tripod",
    "horse": "a black-and-white silhouette of a horse",
    "coffee": "a cup of coffee on a saucer",
    "eagle": "a black-and-white photo of an eagle"
}

In [ ]:
original_images = []
images = []
texts = []
plt.figure(figsize=(16, 5))

for name in descriptions:
    caller = getattr(data, name)
    image = caller()
    image = Image.fromarray(image).convert("RGB")

    plt.subplot(2, 4, len(images) + 1)
    plt.imshow(image)
    plt.title(f"{name}\n{descriptions[name]}")
    plt.xticks([])
    plt.yticks([])

    original_images.append(image)
    images.append(preprocess(image))
    texts.append(descriptions[name])

plt.tight_layout()


In [ ]:
print(images[0].shape)

## Building features

We normalize the images, tokenize each text input, and run the forward pass of the model to get the image and text features.

In [ ]:
image_input = torch.tensor(np.stack(images)).to(device)
text_tokens = clip.tokenize(["This is " + desc for desc in texts]).to(device)

In [ ]:
with torch.no_grad():
    image_features = model.encode_image(image_input).float()
    text_features = model.encode_text(text_tokens).float()

In [ ]:
print(image_features.shape)
print(text_features.shape)

## Calculating cosine similarity

We normalize the features and calculate the dot product of each pair.

$$\text{Normalize}(x) = \frac{x}{||x||}$$

$$\text{cos}(x, y) = \frac{x \cdot y}{||x|| ||y||}$$

In [ ]:

# TODO: normalize image_features and text_features
# hint: use torch.norm
image_features = None
text_features = None

# computing cosine similarity between all image features and all text features
# then convert to numpy array
# TODO
similarity = None

In [ ]:
count = len(descriptions)

plt.figure(figsize=(20, 14))
plt.imshow(similarity, vmin=0.1, vmax=0.3)
# plt.colorbar()
plt.yticks(range(count), texts, fontsize=18)
plt.xticks([])
for i, image in enumerate(original_images):
    plt.imshow(image, extent=(i - 0.5, i + 0.5, -1.6, -0.6), origin="lower")
for x in range(similarity.shape[1]):
    for y in range(similarity.shape[0]):
        plt.text(x, y, f"{similarity[y, x]:.2f}", ha="center", va="center", size=12)

for side in ["left", "top", "right", "bottom"]:
  plt.gca().spines[side].set_visible(False)

plt.xlim([-0.5, count - 0.5])
plt.ylim([count + 0.5, -2])

plt.title("Cosine similarity between text and image features", size=20)

# Zero-Shot Image Classification

You can classify images using the cosine similarity (times 100) as the logits to the softmax operation.

In [ ]:
from torchvision.datasets import CIFAR100

cifar100 = CIFAR100('data', transform=preprocess, download=True)

In [ ]:
text_descriptions = [f"This is a photo of a {label}" for label in cifar100.classes]
text_tokens = clip.tokenize(text_descriptions).to(device)

In [ ]:
print(cifar100.classes)

In [ ]:
with torch.no_grad():
    text_features = model.encode_text(text_tokens).float()
    text_features /= text_features.norm(dim=-1, keepdim=True)

text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
top_probs, top_labels = text_probs.cpu().topk(5, dim=-1)

In [ ]:
plt.figure(figsize=(16, 16))

for i, image in enumerate(original_images):
    plt.subplot(4, 4, 2 * i + 1)
    plt.imshow(image)
    plt.axis("off")

    plt.subplot(4, 4, 2 * i + 2)
    y = np.arange(top_probs.shape[-1])
    plt.grid()
    plt.barh(y, top_probs[i])
    plt.gca().invert_yaxis()
    plt.gca().set_axisbelow(True)
    plt.yticks(y, [cifar100.classes[index] for index in top_labels[i].numpy()])
    plt.xlabel("probability")

plt.subplots_adjust(wspace=0.5)
plt.show()

#### Try your own images

In [ ]:
!wget https://heronscrossing.vet/wp-content/uploads/Golden-Retriever-2048x1365.jpg
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Orange_tabby_cat_sitting_on_fallen_leaves-Hisashi-01A.jpg/224px-Orange_tabby_cat_sitting_on_fallen_leaves-Hisashi-01A.jpg

In [ ]:
# load image and get image feature
orig_image = Image.open("Golden-Retriever-2048x1365.jpg").convert("RGB")

# TODO
# preprocess image, then convert to shape (1, 3, 224, 224) and move it to device
image = None


# TODO
# extract image features and normalize
with torch.no_grad():
    image_features = None


print(image_features.shape)

In [ ]:
# text features
classes = ['dog', 'cat']
text_descriptions = [f"This is a photo of a {label}" for label in classes]

# TODO
# tokenize and move to device
text_tokens = None

# TODO
# extract text features and normalize
with torch.no_grad():
    text_features = None

print(text_features.shape)

In [ ]:
# TODO
# compute probability for each class
class_probs = None

In [ ]:
print(class_probs)

In [ ]:
print('Classification Result')
for ix, label in enumerate(classes):
    print(f"{label}: {class_probs[0][ix] * 100:.2f}%")

In [ ]:
# feel free to try your own image
# and also try changing classes

## Evaluating CLIP on Zero-Shot Image Classification

In [ ]:
# We will evaluate CLIP on conventional image classification dataset (CIFAR10)

from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from tqdm import tqdm

cifar10 = CIFAR10('data', transform=preprocess, download=True, train=False)

test_loader = DataLoader(cifar10, batch_size=100, shuffle=False, num_workers=2)


In [ ]:
print(cifar10.classes)

In [ ]:
# Preparing text features of 10 classes

text_descriptions = [f"This is a photo of a {label}" for label in cifar10.classes]
text_tokens = clip.tokenize(text_descriptions).to(device)

with torch.no_grad():
    text_features = model.encode_text(text_tokens).float()
    text_features /= text_features.norm(dim=-1, keepdim=True)

In [ ]:

correct = 0
total = 0
for images, labels in tqdm(test_loader):
    images, labels = images.to(device), labels.to(device)

    # TODO: extract image features, compute prediction, and compute accuracy
    with torch.no_grad():
        image_features = None

    prediction = None
    correct += None
    total += None

accuracy = correct / total
print(f"Accuracy: {accuracy * 100:.2f}%")

### Prompt Tuning to Improve Zero-Shot Classification

In [ ]:
templates = [
        "a photo of a {c}.",
        "a blurry photo of a {c}.",
        "a black and white photo of a {c}.",
        "a low contrast photo of a {c}.",
        "a high contrast photo of a {c}.",
        "a bad photo of a {c}.",
        "a good photo of a {c}.",
        "a photo of a small {c}.",
        "a photo of a big {c}.",
        "a photo of the {c}.",
        "a blurry photo of the {c}.",
        "a black and white photo of the {c}.",
        "a low contrast photo of the {c}.",
        "a high contrast photo of the {c}.",
        "a bad photo of the {c}.",
        "a good photo of the {c}.",
        "a photo of the small {c}.",
        "a photo of the big {c}."
    ]

In [ ]:


text_features = []
for classname in cifar10.classes:
    text_descriptions = [template.format(c=classname) for template in templates]
    text_tokens = clip.tokenize(text_descriptions).to(device)
    with torch.no_grad():
        class_text_features = model.encode_text(text_tokens).float()
        class_text_features /= class_text_features.norm(dim=-1, keepdim=True)

        class_text_feature = class_text_features.mean(dim=0)
        class_text_feature /= class_text_feature.norm(dim=-1, keepdim=True)
    text_features.append(class_text_feature)

text_features = torch.stack(text_features, dim=0)

In [ ]:

correct = 0
total = 0
for images, labels in tqdm(test_loader):
    images, labels = images.to(device), labels.to(device)

    # TODO: extract image features, compute prediction, and compute accuracy
    with torch.no_grad():
        image_features = None

    prediction = None
    correct += None
    total += None

accuracy = correct / total
print(f"Accuracy: {accuracy * 100:.2f}%")